In [1]:
import altair as alt
import h5py
from pathlib import Path
import numpy as np
import polars as pl
import yaml
from tqdm import tqdm
import pprint
import random
import sys

In [2]:
import altair as alt # for vis code
import numpy as np # for vis code
import polars as pl # for vis code
import random # for vis code
import sys
from datetime import datetime
from neural_feature_identification.dataset_utils import load_project_config, load_session_data, generate_train_test_split, print_data_shape, enforce_samples_as_rows
from neural_feature_identification.vis_utils import make_tidy_norm, make_kinematics_line_plot, make_events_raster_plot, make_features_line_plot
from pathlib import Path

print(sys.executable)
alt.data_transformers.enable("vegafusion")

2025-10-21 17:36:54.700 | INFO     | neural_feature_identification.config:<module>:11 - PROJ_ROOT path is: /uufs/chpc.utah.edu/common/home/u1424875/research-projects/neural-feature-identification-pipeline/src


/uufs/chpc.utah.edu/common/home/u1424875/research-projects/neural-feature-identification-pipeline/.venv/bin/python3


DataTransformerRegistry.enable('vegafusion')

In [3]:
project_config = load_project_config(Path("../config.yaml"))
session_id = "75"
session_dirpath = Path(project_config['paths']['scratch_root']) / session_id # Path("C:/Users/f0397/Desktop/75")
session_data = load_session_data(session_dirpath, project_config)
enforce_samples_as_rows(session_data)
print_data_shape(session_data)

Loading session data: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 10.54it/s]

events: <class 'dict'>
	trial_start_idxs: <class 'numpy.ndarray'>, (1, 140)
	trial_stop_idxs: <class 'numpy.ndarray'>, (1, 140)
kinematics: <class 'dict'>
	kinematics: <class 'numpy.ndarray'>, (12, 13586)
	nip_time: <class 'numpy.ndarray'>, (1, 13586)
nfr: <class 'dict'>
	features: <class 'numpy.ndarray'>, (192, 13586)
sbp-raw: <class 'dict'>
	features: <class 'numpy.ndarray'>, (192, 13586)
dwt-db4: <class 'dict'>
	features: <class 'numpy.ndarray'>, (1920, 13586)
mav: <class 'dict'>
	features: <class 'numpy.ndarray'>, (192, 13586)
events: <class 'dict'>
	trial_start_idxs: <class 'numpy.ndarray'>, (140, 1)
	trial_stop_idxs: <class 'numpy.ndarray'>, (140, 1)
kinematics: <class 'dict'>
	kinematics: <class 'numpy.ndarray'>, (13586, 12)
	nip_time: <class 'numpy.ndarray'>, (13586, 1)
nfr: <class 'dict'>
	features: <class 'numpy.ndarray'>, (13586, 192)
sbp-raw: <class 'dict'>
	features: <class 'numpy.ndarray'>, (13586, 192)
dwt-db4: <class 'dict'>
	features: <class 'numpy.ndarray'>, (13586, 1

In [4]:
kinematics_data = session_data['kinematics']['kinematics']
kinematics_timestamps = session_data['kinematics']['nip_time']
start_timestamps = session_data['events']['trial_start_idxs']
stop_timestamps = session_data['events']['trial_stop_idxs']
split_params = project_config['analysis']['train_test_split']
kinematics_data.shape

(13586, 12)

In [5]:
train_idxs, test_idxs, train_trials_info, test_trials_info =  generate_train_test_split(
    kinematics=kinematics_data,
    kinematics_timestamps=kinematics_timestamps,
    trial_start_timestamps=start_timestamps,
    trial_stop_timestamps=stop_timestamps,
    train_ratio=split_params['train_ratio'],
    training_type=split_params['split_type'],
    include_combined=split_params['include_combined'],
)

2025-10-21 17:37:11.328 | INFO     | neural_feature_identification.dataset_utils:generate_train_test_split:117 - ---Generating training and test sets---
2025-10-21 17:37:11.328 | INFO     | neural_feature_identification.dataset_utils:generate_train_test_split:123 - Converting trial timestamps to array indices...
2025-10-21 17:37:11.329 | INFO     | neural_feature_identification.dataset_utils:generate_train_test_split:140 - Found 140 valid trials
2025-10-21 17:37:11.334 | INFO     | neural_feature_identification.dataset_utils:generate_train_test_split:174 - Ignoring combined movements. Processing 120 trials...
2025-10-21 17:37:11.334 | INFO     | neural_feature_identification.dataset_utils:generate_train_test_split:183 - Gesture representations:
2025-10-21 17:37:11.334 | INFO     | neural_feature_identification.dataset_utils:generate_train_test_split:185 - - dof_1_pos: 10 trials
2025-10-21 17:37:11.335 | INFO     | neural_feature_identification.dataset_utils:generate_train_test_split:18

In [6]:
vis_df = make_tidy_norm(session_data, project_config)

Original data has 13586 timestamps
Resampling data by averaging over 20384.84 timestamp units
Resampled data has 661 timestamps


In [9]:
x_domain = [vis_df['timestamps'].min(), vis_df['timestamps'].max()]

start_stamps = session_data.get('events', {}).get('trial_start_idxs', np.array([]))
stop_stamps = session_data.get('events', {}).get('trial_stop_idxs', np.array([]))

random.seed(42)
selected_channels_map = {}
dwt_subsample_count = project_config['vis']['dwt_subsample_count']
dwt_df = vis_df.filter(pl.col('feature_type') == 'DWT-DB4')
dwt_channels_all = dwt_df.get_column('feature_id').unique().to_list()
if len(dwt_channels_all) > dwt_subsample_count:
    selected_channels_map['dwt-db4'] = random.sample(dwt_channels_all, dwt_subsample_count)

In [ ]:
plt_kinematics = make_kinematics_line_plot(vis_df, project_config, x_domain=x_domain)
plt_events = make_events_raster_plot(start_stamps, stop_stamps, x_domain=x_domain)
plt_features = []
for f_name in project_config['analysis']['feature_sets']:
    feature_key = f_name.lower()
    selected_channels = selected_channels_map.get(feature_key)
    plt = make_features_line_plot(vis_df, f_name, selected_channels, x_domain=x_domain)
    plt_features.append(plt)
final_chart = alt.vconcat(
    plt_kinematics,
    plt_events,
    *plt_features,
).resolve_scale(
    color='independent',
).properties(
)

In [11]:
final_chart.save(f"labeled_features_vis-{datetime.now().strftime("%Y%m%d_%H%M%S")}.html")

ImportError: The "vegafusion" data transformer and chart.transformed_data feature requires
version 1.5.0 or greater of the 'vegafusion-python-embed' and 'vegafusion' packages.
These can be installed with pip using:
    pip install "vegafusion[embed]>=1.5.0"
Or with conda using:
    conda install -c conda-forge "vegafusion-python-embed>=1.5.0" "vegafusion>=1.5.0"

ImportError: No module named 'vegafusion'